# Prueba - Python para el análisis de datos
## Tomás Marín

En esta prueba validaremos nuestros conocimientos sobre Python. Para lograrlo,
necesitarás aplicar lo aprendido en las unidades anteriores.
Lee todo el documento antes de comenzar el desarrollo individual, para asegurarte de tener
el máximo de puntaje y enfocar bien los esfuerzos.


Prerrequisitos
1. Para realizar lo solicitado debes crear la base de datos classicmodels en tu motor PostgreSQL. Para esto, abre una ventana de terminal y ejecuta la siguiente instrucción:
   psql -h localhost -p 5432 -U postgres -c "CREATE DATABASE classicmodels;"
2. Una vez creada la base de datos, debes importar el archivo classicmodels.sql a esta
base de datos.
   psql -h localhost -p 5432 -U postgres -d classicmodels -f classicmodels.sql



 Abrir CMD en sistema
 
 psql -h localhost -p 5432 -U postgres -c "CREATE DATABASE classicmodels;"
 Definir ruta del archivo
- cd C:\Users\Legion\Documents\desafio_latam\Python\Prueba

Conectarde a la base classicmodels
- C:\Users\Legion\Documents\desafio_latam\Python\Prueba>psql -h localhost -U postgres -d classicmodels

Importar el archivo dentro de SQL
- classicmodels=# \i 'classicmodels.sql'

Comprobar que este todo ok:
- classicmodels=# \dt *.* 


## Descripción
El área comercial de una empresa pide realizar un cierre de año de las ventas, tanto para
revisar si las metas fueron cumplidas, como para poder planificar el siguiente año. Para ello,
considerarán los datos del dataset classicmodels.sql para responder algunas preguntas,
realizando las siguientes tareas.

1. Genera una función llamada leer_tabla(tabla, engine) y utilízala para leer tablas
completas desde la base de datos en dataframes independientes. Utilizando esta
función, importa las siguientes tablas:
- order
- orderdetails
- customers
- products
- employees

In [7]:
import pandas as pd
import sqlalchemy
import psycopg2

In [8]:
from sqlalchemy import create_engine

#Completar los datos necesarios para conectar la database de postgres
usuario = "postgres"
contraseña = "postgres"  
host = "localhost"
puerto = "5432"
base_datos = "classicmodels"

# Crear el engine de conexión
engine = create_engine(f"postgresql://{usuario}:{contraseña}@{host}:{puerto}/{base_datos}")



In [9]:
def leer_tabla(tabla, engine):
    """
    Lee una tabla completa desde PostgreSQL y la devuelve como DataFrame de pandas.
    """
    query = f"SELECT * FROM {tabla};"
    return pd.read_sql(query, engine)


In [10]:
orders = leer_tabla('orders', engine)
orderdetails = leer_tabla('orderdetails', engine)
customers = leer_tabla('customers', engine)
products = leer_tabla('products', engine)
employees = leer_tabla('employees', engine)


2. Realiza el cruce entre los DataFrames, asegurándote de utilizar correctamente el
parámetro validate para asegurar la integridad referencial



In [12]:
# "Orders es la hoja principal que posee informacion vinculada a las otras pestañas
# Identificar la columna  en comun con todos los dataframes para unirlas
common_columns = set(orders.columns) & set(customers.columns) #Interseccion de nombres de columnas en un set
print("La columna comun es:", common_columns)

common_columns = set(orders.columns) & set(orderdetails.columns) #Interseccion de nombres de columnas en un set
print("La columna comun es:", common_columns)

common_columns = set(products.columns) & set(orderdetails.columns) #Interseccion de nombres de columnas en un set
print("La columna comun es:", common_columns)

common_columns = set(employees.columns) & set(products.columns) #Interseccion de nombres de columnas en un set
print("La columna comun es:", common_columns)

La columna comun es: {'customerNumber'}
La columna comun es: {'orderNumber'}
La columna comun es: {'productCode'}
La columna comun es: set()


Al indagar en los nombres de las columnas de cada df se obtiene el nexo sobre el cual realizar el cruce de las tablas

customers > order > (customerNumber)\
orders -> orderdetails (orderNumber)\
orderdetails -> products (productCode)\

Pero employees no tiene columna común con ningun otro dataframe. Por eso se buscó individualmente algun campo similar a 'employeeNumber' y se encontró 'salesRepEmployeeNumber' en el df de customers. 

In [14]:
df_completo = customers.merge(
orders, on="customerNumber", how="inner", validate="1:m" # Un cliente puede hacer muchos pedidos
).merge(
orderdetails, on="orderNumber", how="inner", validate="1:m" # Una orden tiene muchos detalles o productos asociados
).merge(
products, on="productCode", how="inner", validate="m:1" # Se pueden hacer muchas compras de un producto único
).merge(
employees, left_on="salesRepEmployeeNumber", right_on="employeeNumber", how="left", validate="m:1" # Muchos clientes o fueron atendidos por un empleado
)

3. Agrega las siguientes columnas, considerando su nombre y la fórmula asociada
- venta: quantityOrdered*priceEach
- costo: quantityOrdered*buyPrice
- ganancia: considerando las columnas anteriores


In [16]:
# Crear nuevos campos a partir de forulas proporcionadas
df_completo["venta"] = df_completo["quantityOrdered"] * df_completo["priceEach"]
df_completo["costo"] = df_completo["quantityOrdered"] * df_completo["buyPrice"]
df_completo["ganancia"] = df_completo["venta"] - df_completo["costo"]

4. ¿Cuál fue el total de ventas por línea de productos? Incluye una fila de totales.

In [18]:
# Crear una tabla pivote con la funcion de suma de las ventas
pivot_ventas_por_linea = pd.pivot_table(
    df_completo,
    index="productLine",   # Agrupar por valores unicos          
    values="venta",        # valores a sumar   
    aggfunc="sum",    
    margins=True,       #crear nuevo campo de totales
    margins_name="Total_ventas" #nombre de nuevo campo
)

print("Ventas totales por líneas de productos:")
print(pivot_ventas_por_linea)


Ventas totales por líneas de productos:
                       venta
productLine                 
Classic Cars      3853922.49
Motorcycles       1121426.12
Planes             954637.54
Ships              663998.34
Trains             188532.92
Trucks and Buses  1024113.57
Vintage Cars      1797559.63
Total_ventas      9604190.61


5. ¿Cuántos clientes distintos hicieron compras?

In [20]:
# df_completo solo posee datos de los clientes que realizaron compras
# Contar valores unicos de clientes que realizaron compras
clientes_distintos = df_completo["customerNumber"].nunique()
print("Número de clientes distintos que hicieron compras:", clientes_distintos)


Número de clientes distintos que hicieron compras: 98


6. ¿Existen clientes que aún no han hecho ninguna compra? ¿Cuántos son?


In [22]:
clientes_totales = len(customers["customerNumber"]) #Total de clientes registrados
clientes_sin_compra = clientes_totales - clientes_distintos # Total de clientes menos los que han realizado compras
print("Número de clientes que no han hecho compras:", clientes_sin_compra)

Número de clientes que no han hecho compras: 24


7. Se solicita la creación de dos reportes, que respondan las preguntas dadas

● ¿Cuáles fueron los 10 clientes que reportan mayores ventas brutas en dinero durante
el año 2005? Genera un DataFrame y guárdalo en una tabla de Postgre llamada
top_10_clientes_2005, en la que se especifique el nombre del cliente y su
correspondiente venta, costo y ganancia.

● ¿Cuál fue el top 10 de artículos más vendidos durante el año 2005? Genera un
DataFrame y guárdalo en una tabla de Postgre llamada top_10_productos_2005, en la
que se especifique el nombre del producto y su correspondiente venta, costo y
ganancia


In [24]:
df_completo.info() #Identificar las columnas de interes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2996 entries, 0 to 2995
Data columns (total 42 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   customerNumber          2996 non-null   int64  
 1   customerName            2996 non-null   object 
 2   contactLastName         2996 non-null   object 
 3   contactFirstName        2996 non-null   object 
 4   phone                   2996 non-null   object 
 5   addressLine1            2996 non-null   object 
 6   addressLine2            592 non-null    object 
 7   city                    2996 non-null   object 
 8   state                   1337 non-null   object 
 9   postalCode              2823 non-null   object 
 10  country                 2996 non-null   object 
 11  salesRepEmployeeNumber  2996 non-null   float64
 12  creditLimit             2996 non-null   float64
 13  orderNumber             2996 non-null   int64  
 14  orderDate               2996 non-null   

In [25]:
from funciones import filtrar_por_fecha, generar_reporte, guardar_en_postgres


In [27]:
# Filtrar datos del año 2005
df_2005 = filtrar_por_fecha(df_completo, "orderDate", "2005-01-01", "2005-12-31")
df_2005.head(5)

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,...,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,venta,costo,ganancia
116,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",None,Nantes,None,44000,...,Hernandez,Gerard,x2028,ghernande@classicmodelcars.com,4,1102.0,Sales Rep,1607.76,1025.01,582.75
117,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",None,Nantes,None,44000,...,Hernandez,Gerard,x2028,ghernande@classicmodelcars.com,4,1102.0,Sales Rep,8317.80,4290.30,4027.50
118,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",None,Nantes,None,44000,...,Hernandez,Gerard,x2028,ghernande@classicmodelcars.com,4,1102.0,Sales Rep,7380.38,3247.23,4133.15
119,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",None,Nantes,None,44000,...,Hernandez,Gerard,x2028,ghernande@classicmodelcars.com,4,1102.0,Sales Rep,1541.69,858.36,683.33
120,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",None,Nantes,None,44000,...,Hernandez,Gerard,x2028,ghernande@classicmodelcars.com,4,1102.0,Sales Rep,1205.20,942.00,263.20


In [28]:

# Reporte 1: Top 10 clientes 2005

top_clientes_2005 = generar_reporte( #Utilizamos la funcion generar reporte
    df=df_2005,
    filas_index=["customerName"],
    valores=["venta", "costo", "ganancia"],
    medida="sum"
).sort_values("venta", ascending=False).head(10)

guardar_en_postgres(top_clientes_2005, "top_10_clientes_2005", engine)




✅ Tabla 'top_10_clientes_2005' guardada en PostgreSQL con 10 registros.


In [29]:
top_clientes_2005

,customerName,costo,ganancia,venta
12,Euro+ Shopping Channel,169989.97,120028.55,290018.52
27,Mini Gifts Distributors Ltd.,115084.72,77397.01,192481.73
21,La Rochelle Gifts,55527.04,35620.07,91147.11
39,The Sharp Gifts Warehouse,50843.02,33141.87,83984.89
10,"Down Under Souveniers, Inc",46389.52,28630.61,75020.13
1,"Anna's Decorations, Ltd",35414.90,21517.40,56932.30
33,Salzburg Collectables,33536.26,18883.81,52420.07
16,Gifts4AllAges.com,33221.25,17585.60,50806.85
7,Corporate Gift Ideas Co.,28561.31,18220.35,46781.66
28,"Oulu Toy Supplies, Inc.",27493.61,19276.91,46770.52


In [30]:
# Reporte 2: Top 10 productos 2005

top_productos_2005 = generar_reporte( 
    df=df_2005,
    filas_index=["productName"],
    valores=["venta", "costo", "ganancia"],
    medida="sum"
).sort_values("venta", ascending=False).head(10)

guardar_en_postgres(top_productos_2005, "top_10_productos_2005", engine)

✅ Tabla 'top_10_productos_2005' guardada en PostgreSQL con 10 registros.


In [31]:
top_productos_2005

,productName,costo,ganancia,venta
76,1992 Ferrari 360 Spider red,27031.30,25946.98,52978.28
91,2003 Harley-Davidson Eagle Drag Bike,19023.18,15832.94,34856.12
36,1952 Alpine Renault 1300,17152.92,16884.28,34037.20
55,1968 Ford Mustang,17161.20,14538.31,31699.51
39,1956 Porsche 356A Coupe,25066.50,6365.64,31432.14
89,2002 Suzuki XREO,15308.37,15125.72,30434.09
58,1969 Dodge Charger,15974.56,13592.71,29567.27
83,1997 BMW R 1100 S,16493.06,12254.63,28747.69
9,1917 Grand Touring Sedan,16386.30,11444.40,27830.70
67,1972 Alfa Romeo GTA,18678.24,8547.60,27225.84


# funciones.py

import pandas as pd


# 1. Función para filtrar por fechas. Sirve para realizar reportes en dentro de ciertas franjas temporales

def filtrar_por_fecha(df, columna_fecha, fecha_inicio, fecha_fin):
    """
    Filtra un DataFrame entre un rango de fechas dado.
    """
    df[columna_fecha] = pd.to_datetime(df[columna_fecha]) # Convertir a datetime el objeto
    return df[(df[columna_fecha] >= fecha_inicio) & (df[columna_fecha] <= fecha_fin)] # Filtra valores mayores o igual a fecha inicio y menor o igual a fecha final



# 2. Función para generar reportes como tablas pivote con a partir de un data frame con informacion de ventas. Por default calcula totales

def generar_reporte(df, filas_index, columnas=None, valores=None, medida="sum"):
    """
    Genera un reporte en formato de tabla pivote.
    """
    return pd.pivot_table(
        df,
        index=filas_index, # Valores a partir de los cuales se quiere agrupar
        columns=columnas, 
        values=valores, # valores que se quieren calcular
        aggfunc=medida, # default de medida = suma
        fill_value=0
    ).reset_index()



# 3. Función para guardar en PostgreSQL. Se utiliza para guardar reporte como una nueva tabla en SQL

def guardar_en_postgres(df, nombre_tabla, engine, if_exists="replace"):
    """
    Guarda un DataFrame en PostgreSQL.
    """
    df.to_sql(nombre_tabla, engine, if_exists=if_exists, index=False)
    print(f"✅ Tabla '{nombre_tabla}' guardada en PostgreSQL con {len(df)} registros.")
